In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import csv
import re
import math

#Open a blank csv
csv_file = open('courses_whole3.csv', 'w')

writer = csv.writer(csv_file)

#Create column headers for the variables
writer.writerow(['title', 'partner', 'rating', 'enrollment', 
                 'subject', 'skills', 'duration', 'level', 
                 'new_career', 'promotion'])

84

In [27]:
#This cell scrapes the page urls for each separate page, which are then used in the cell below this to scrape 
#the information from each individual page

driver = webdriver.Chrome()

#Main Coursera page filtered for courses conducted in English only
driver.get('https://www.coursera.org/search?index=prod_all_products_term_optimization&allLanguages=English&entityTypeDescription=Courses&productDifficultyLevel=Beginner')

#Scrape the total number of courses after being filtered
num_classes_str = driver.find_element_by_xpath('//h3[@class="rc-NumberOfResultsSection body-2-text"]').text

#Convert number of classes to an integer
num_classes = int((re.findall(r'\d+', num_classes_str))[0])

#Initialize page number to 1
current_page = 1  

#Calculate total number of pages
num_pages = math.ceil(num_classes / 10)

#Course urls
course_urls4 = []

# Initiate while loop to scroll through all pages
while current_page <= num_pages:
    time.sleep(2)
    a_tags = driver.find_elements_by_xpath('//li[@class="ais-InfiniteHits-item"]/div/a')
    hrefs = [a_tag.get_attribute('href') for a_tag in a_tags]
    course_urls4.extend(hrefs)
    current_page += 1
    driver.execute_script("window.scrollTo(0, 1500);")
    button = driver.find_element_by_xpath('//button[@aria-label="Next Page"]')
    button.click()



In [ ]:
#Title of course
def get_title(driver):
    try:
        title = driver.find_element_by_xpath('//h1[@class="banner-title m-b-0 banner-title-without--subtitle"]').text
    except:
        title = 'Missing'
    finally:
        return title 

#Partner sponsoring the course
def get_partner(driver):
    try:
#         title = driver.find_element_by_xpath('//div[@class = "m-b-1s m-r-1img"]//img.get_atttribute("title")').text
        title = driver.find_element_by_xpath('//img[@class="_1g3eaodg"]').get_attribute('title')
    except:
        title = 'Missing'
    finally:
        return title

#Course rating on scale of 0-5    
def get_rating(driver):
    try:
        rating = driver.find_element_by_xpath('//span[@class="_16ni8zai m-b-0 rating-text number-rating number-rating-expertise"]').text
    except:
        rating = 'Missing'
    finally:
        return rating

#Total number of people who previously enrolled
def get_enrollment(driver):
    try:
        enrollment = driver.find_element_by_xpath('//div[@class="_3vb6hs"]/span/strong').text
    except:
        enrollment = 'Missing'
    finally:
        return enrollment

#Subject of the course
def get_subject(driver):
    try:
        subject = driver.find_element_by_xpath('//div[@class="_e0mvswb"]/div[2]/a').text
    except:
        subject = 'Missing'
    finally:
        return subject

#Skills acquired by taking the course
def get_skills(driver):
    try:
#         skills = driver.find_element_by_xpath('//div[@class="Skills p-x-2 p-t-1 p-b-2 skills-sdp-content-exp border-a"][@class="content-inner"]').text
        skills = driver.find_elements_by_xpath('//span[@class="_1q9sh65"]')
        skills = [skill.text for skill in skills]
    except:
        skills = 'Missing'
        print(f'Missing "skills" at {driver.url}')
    finally:
        return skills

# #Duration of the course
# def get_duration(driver):
#     try:
# #         duration = driver.find_element_by_xpath('//div[@class="_1tu07i3a"]/h4').text
#         duration = driver.find_element_by_xpath('//div[@class="ProductGlance"]/div[4]').text
#     except:
#         duration = 'Missing'
#     finally:
#         return duration

#Course difficulty level: beginner, intermediate, or advanced
def get_level_and_duration(driver):
    try:
        temp = driver.find_element_by_xpath('//div[@class="ProductGlance"]/div[4]').text
        if temp.lower().find('level') == -1:
            duration = temp
            level = 'Missing'
        else:
            level = temp
            duration = driver.find_element_by_xpath('//div[@class="ProductGlance"]/div[5]').text
    except:
        level = 'Missing'
        duration = 'Missing'
        print(f'Duration and level missing at {driver.url}')
    finally:
        return level, duration

#Percent of people who started a new career after taking the course
def get_newcareer_and_promotion(driver):
    try:
#         new_career = driver.find_element_by_xpath('//div[@class="LearnerOutcomes__text-wrapper"]/div[@class="m-l-1s m-r-1 m-b-0 LearnerOutcomes__percent"]').text
        percents = driver.find_elements_by_xpath('//div[@class="m-l-1s m-r-1 m-b-0 LearnerOutcomes__percent"]')
        percents = list(filter(lambda s: s != '', [percent.text for percent in percents]))
        newcareer, promotion = percents
    except Exception as e:
        print(type(e), e)
        percents = ('Missing', 'Missing')
        newcareer, promotion = percents
        print(f'Missing newcareer and promotion at page {driver.url}')  
    finally:
        return newcareer, promotion

# #Percent of people who got a raise or promotion after 
# def get_promotion(driver):
#     try:
#         promotion = driver.find_element_by_xpath('//div[@class="_jyhj5r"]//h2[@class ="_1l2q8kho m-x-1 m-b-0"]/h2').text
#     except:
#         promotion = 'Missing'
#     finally:
#         return promotion
    

#For loop which goes through each page url scraped in the above cell and places information in an empty 
#dictionary named course_dict
driver = webdriver.Chrome()
for url in course_urls:
    course_dict = {}
    driver.get(url)
    course_dict['title'] = get_title(driver)
    course_dict['partner'] = get_partner(driver)
    course_dict['rating'] = get_rating(driver)
    course_dict['enrollment'] = get_enrollment(driver)
    course_dict['subject'] = get_subject(driver)
    course_dict['skills'] = get_skills(driver)
    course_dict['level'], course_dict['duration'] = get_level_and_duration(driver)
    course_dict['new_career'], course_dict['promotion'] = get_newcareer_and_promotion(driver)
    writer.writerow(course_dict.values())
    print(course_dict)
    
#Close the csv file
driver.close()
csv_file.close()

In [42]:
import pandas as pd
df = pd.read_csv('courses_whole3.csv')

In [43]:
df

,title,partner,rating,enrollment,subject,skills,duration,level,new_career,promotion
0,AstroTech: The Science and Technology behind A...,The University of Edinburgh,4.7\nstars,"7,418",Physical Science and Engineering,[],Beginner Level,Approx. 6 hours to complete,Missing,Missing
1,Visual Analytics with Tableau,"University of California, Davis",4.6\nstars,"24,220",Data Science,"['Visual Analytics', 'Map', 'Tableau Software'...",Missing,Flexible deadlines\nReset deadlines in accorda...,Missing,Missing
2,Python Data Analysis,Rice University,4.7\nstars,"14,515",Computer Science,[],Missing,Flexible deadlines\nReset deadlines in accorda...,Missing,Missing
3,صياغة الأفكار المحفِّزة على الابتكار,University of Virginia,4.9\nstars,"1,887",Business,[],Beginner Level\nNo prior experience needed to ...,Approx. 6 hours to complete,Missing,Missing
4,FinTech Disruptive Innovation: Implications fo...,The Hong Kong University of Science and Techno...,4.6\nstars,Missing,Business,[],Missing,Flexible deadlines\nReset deadlines in accorda...,Missing,Missing
...,...,...,...,...,...,...,...,...,...,...
2723,Philosophy and the Sciences: Introduction to t...,The University of Edinburgh,4.6\nstars,Missing,Arts and Humanities,[],Missing,Approx. 10 hours to complete,Missing,Missing
2724,International Hospitality & Healthcare Service...,Yonsei University,4.6\nstars,Missing,Business,"['Variable Pricing', 'Marketing', 'Hospital Ma...",Missing,Flexible deadlines\nReset deadlines in accorda...,Missing,Missing
2725,Aboriginal Worldviews and Education,University of Toronto,4.7\nstars,Missing,Social Sciences,[],Missing,Approx. 15 hours to complete,Missing,Missing
2726,Multimodal Literacies: Communication and Learn...,University of Illinois at Urbana-Champaign,4.9\nstars,Missing,Social Sciences,[],Missing,Approx. 35 hours to complete,Missing,Missing
